In [ ]:
!git clone https://github.com/KinhNguyenVan/vietocr.git

%cd vietocr

!pip install --quiet -r /kaggle/working/vietocr/vietocr/requirement.txt

! pip install gdown

!pip install git+https://github.com/JaidedAI/EasyOCR.git

In [ ]:
import cv2

import numpy as np

import os

from PIL import Image

import matplotlib.pyplot as plt

from vietocr.tool.predictor import Predictor

from vietocr.tool.config import Cfg
import easyocr
from PIL import Image 
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



config = Cfg.load_config_from_name('vgg_transformer')

config['cnn']['pretrained'] = False

config['device'] = device

config['predictor']['beamsearch'] = False

vietocr_predictor = Predictor(config)
reader = easyocr.Reader(['vi','en'],gpu=True) # this needs to run only once to load the model into memory

In [ ]:
with open(".../data/...json","r") as f:
    test=json.load(f)
data = dict()

img_dir = '..../image_dir'  
for i in range(len(test)):

    img_id = test[str(i)]['image']

    img_path = os.path.join(img_dir, img_id)

    

    # Load the image using OpenCV

    img = cv2.imread(img_path)

    

    if img is None:

        raise FileNotFoundError(f"Image not found or failed to load: {img_path}")

    

    # Convert image to tensor and move to GPU

    img_tensor = torch.from_numpy(img).to(device)

    detection_results=reader.readtext(img_path)


    fulltext = ""

    try:

        for line in detection_results:

            box = line[0]  # Bounding box coordinates

            xmin, ymin = int(min([point[0] for point in box])), int(min([point[1] for point in box]))

            xmax, ymax = int(max([point[0] for point in box])), int(max([point[1] for point in box]))

            

            # Crop detected text area (move tensor back to CPU for cropping)

            cropped_img = img_tensor.cpu().numpy()[ymin:ymax, xmin:xmax]

            cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))

            # Recognize text using VietOCR

            recognized_text = vietocr_predictor.predict(cropped_img_pil)

            fulltext += recognized_text.lower() + " "

    except:

        fulltext=""



    image_data = {

        "image": img_id,

        "ocr": fulltext

    }

    data[str(i)] = image_data

    if(i%10==0):

        print(i,fulltext)



In [ ]:
output_file=".../...json"
with open(output_file, 'w', encoding='utf-8') as json_file:  
    json.dump(data, json_file, ensure_ascii=False, indent=4) 